In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
run_name = input()

 architectures


In [3]:
# System imports
import os
import sys
import yaml

# External imports
import matplotlib.pyplot as plt
import scipy as sp
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.metrics import auc
import numpy as np
import pandas as pd
import seaborn as sns
import torch
from pytorch_lightning.loggers import TensorBoardLogger, WandbLogger
from pytorch_lightning import Trainer
import frnn
import wandb
import math
sys.path.append('../..')

from LightningModules.DualEmbedding.Models.vanilla_dual_embedding import VanillaDualEmbedding

device = "cuda" if torch.cuda.is_available() else "cpu"
from pytorch_lightning.callbacks import ModelCheckpoint

In [4]:
def kaiming_init(model):
    for name, param in model.named_parameters():
        if name.endswith(".bias"):
            param.data.fill_(0)
        elif name.startswith("layers.0"):  # The first layer does not have ReLU applied on its input
            param.data.normal_(0, 1 / math.sqrt(param.shape[1]))
        else:
            param.data.normal_(0, math.sqrt(2) / math.sqrt(param.shape[1]))

## Sweep

In [5]:
with open("dual_embedding_sweep.yaml") as f:
        sweep_hparams = yaml.load(f, Loader=yaml.FullLoader)
with open("dual_embedding_default.yaml") as f:
        default_hparams = yaml.load(f, Loader=yaml.FullLoader)

In [6]:
sweep_configuration = {
    "name": run_name,
    "project": "ITk_barrell_dual_embedding",
    "metric": {"name": "pur", "goal": "maximize"},
    "method": "grid",
    "parameters": sweep_hparams
}

In [7]:
def load_dict(model):
    checkpoint = torch.load("/global/cfs/cdirs/m3443/usr/ryanliu/ITk_embedding/ITk_dual_embedding/3ijb4qnw/checkpoints/last.ckpt")
    state_dict = checkpoint["state_dict"]
    names = [i for i in state_dict]
    for i in names:
        state = state_dict[i]
        i = i.replace("input_layer1", "input_layer2")
        i = i.replace("layers1", "layers2")
        i = i.replace("output_layer1", "output_layer2")
        state_dict[i] = state

    model.load_state_dict(state_dict)
    del state_dict
    del checkpoint
    return model

In [8]:
def training():
    wandb.init()
    model = VanillaDualEmbedding({**default_hparams, **wandb.config})
    
    if model.hparams["use_dual_encoder"]:
        model = load_dict(model)
    else:
        kaiming_init(model)
    
    checkpoint_callback = ModelCheckpoint(
        monitor='pur',
        mode="max",
        save_top_k=2,
        save_last=True)

    logger = WandbLogger()
    trainer = Trainer(gpus=1, max_epochs=default_hparams["max_epochs"], log_every_n_steps = 50, logger=logger, callbacks=[checkpoint_callback], default_root_dir="/global/cfs/cdirs/m3443/usr/ryanliu/ITk_dual_embedding/")
    trainer.fit(model)

In [ ]:
sweep_id = wandb.sweep(sweep_configuration, project = "ITk_barrel_dual_embedding")

# run the sweep
wandb.agent(sweep_id, function=training)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Create sweep with ID: 6y0m08n8
Sweep URL: https://wandb.ai/exatrkx/ITk_barrel_dual_embedding/sweeps/6y0m08n8


wandb: Agent Starting Run: hv4fwgw9 with config:
wandb: 	use_bidir_truth: False
wandb: 	use_dual_encoder: True
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: exatrkx (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.11 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/global/homes/r/ryanliu/.conda/envs/gnn/lib/python3.8/site-packages/pytorch_lightning/loggers/wandb.py:341: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(
wandb: WARNING Config item 'use_bidir_truth' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'use_dual_encoder' was locked by 'sweep' (ignored update).

  | Name          | Type             | Params
---------------------------------------------------
0 | cos           | CosineSimilarity | 0     
1 | input_layer1  | Linear           | 12.3 K
2 | layers1       | ModuleList       | 5.2 M 
3 | output_layer1 | Linear           | 16.4 K
4 | input_layer2  | Linear           | 12.3 K
5 | layers2       | M

Validation sanity check:   0%|          | 0/2 [00:00<?, ?it/s]

/global/homes/r/ryanliu/.conda/envs/gnn/lib/python3.8/site-packages/pytorch_lightning/trainer/data_loading.py:116: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 256 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Validation sanity check:  50%|█████     | 1/2 [00:01<00:01,  1.60s/it]

/global/homes/r/ryanliu/.conda/envs/gnn/lib/python3.8/site-packages/pytorch_lightning/utilities/data.py:59: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 5. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  warning_cache.warn(


/global/homes/r/ryanliu/.conda/envs/gnn/lib/python3.8/site-packages/pytorch_lightning/trainer/data_loading.py:116: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 256 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Epoch 0:  99%|█████████▉| 1000/1010 [03:56<00:02,  4.22it/s, loss=0.121, v_num=wgw9]
Validating: 0it [00:00, ?it/s]
Epoch 1:  99%|█████████▉| 1000/1010 [04:03<00:02,  4.10it/s, loss=0.119, v_num=wgw9]
Validating: 0it [00:00, ?it/s]
Epoch 2:  99%|█████████▉| 1000/1010 [04:04<00:02,  4.09it/s, loss=0.12, v_num=wgw9] 
Validating: 0it [00:00, ?it/s]
Epoch 3:  99%|█████████▉| 1000/1010 [04:05<00:02,  4.08it/s, loss=0.12, v_num=wgw9] 
Validating: 0it [00:00, ?it/s]
Epoch 4:  99%|█████████▉| 1000/1010 [04:06<00:02,  4.06it/s, loss=0.12, v_num=wgw9]
Validating: 0it [00:00, ?it/s]
Epoch 5:  99%|█████████▉| 1000/1010 [04:05<00:02,  4.07it/s, loss=0.118, v_num=wgw9]
Validating: 0it [00:00, ?it/s]
Epoch 6:  99%|█████████▉| 1000/1010 [04:07<00:02,  4.04it/s, loss=0.119, v_num=wgw9]
Validating: 0it [00:00, ?it/s]
Epoch 7:  99%|█████████▉| 1000/1010 [04:09<00:02,  4.01it/s, loss=0.12, v_num=wgw9] 
Validating: 0it [00:00, ?it/s]
Epoch 8:  76%|███████▌  | 765/1010 [03:09<01:00,  4.03it/s, loss=0.119, v

## Construct PyLightning model

In [4]:
with open("dual_embedding_default.yaml") as f:
    hparams = yaml.load(f, Loader=yaml.FullLoader)

In [5]:
model = VanillaDualEmbedding(hparams)

## Metric Learning

In [6]:
from pytorch_lightning.callbacks import ModelCheckpoint

checkpoint_callback = ModelCheckpoint(
    monitor='pur',
    mode="max",
    save_top_k=2,
    save_last=True)

In [ ]:
kaiming_init(model)
logger = WandbLogger(project="ITk_dual_embedding")
trainer = Trainer(gpus=1, max_epochs=hparams["max_epochs"], logger=logger, num_sanity_val_steps=2, callbacks=[checkpoint_callback], log_every_n_steps = 50, default_root_dir="/global/cfs/cdirs/m3443/usr/ryanliu/ITk_embedding/")
trainer.fit(model)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: exatrkx (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.11 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade



  | Name          | Type             | Params
---------------------------------------------------
0 | cos           | CosineSimilarity | 0     
1 | input_layer1  | Linear           | 12.3 K
2 | layers1       | ModuleList       | 5.2 M 
3 | output_layer1 | Linear           | 16.4 K
4 | input_layer2  | Linear           | 12.3 K
5 | layers2       | ModuleList       | 5.2 M 
6 | output_layer2 | Linear           | 16.4 K
7 | act           | GELU             | 0     
---------------------------------------------------
10.6 M    Trainable params
0         Non-trainable params
10.6 M    Total params
42.214    Total estimated model params size (MB)


Validation sanity check:   0%|          | 0/2 [00:00<?, ?it/s]

/global/homes/r/ryanliu/.conda/envs/gnn/lib/python3.8/site-packages/pytorch_lightning/trainer/data_loading.py:116: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 256 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Validation sanity check:  50%|█████     | 1/2 [00:02<00:02,  2.63s/it]

/global/homes/r/ryanliu/.conda/envs/gnn/lib/python3.8/site-packages/pytorch_lightning/utilities/data.py:59: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 5. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  warning_cache.warn(


/global/homes/r/ryanliu/.conda/envs/gnn/lib/python3.8/site-packages/pytorch_lightning/trainer/data_loading.py:116: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 256 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Epoch 0:  99%|█████████▉| 1000/1010 [04:17<00:02,  3.88it/s, loss=0.0983, v_num=q6d8]
Validating: 0it [00:00, ?it/s]
Epoch 1:  99%|█████████▉| 1000/1010 [03:50<00:02,  4.34it/s, loss=0.102, v_num=q6d8] 
Validating: 0it [00:00, ?it/s]
Epoch 2:  99%|█████████▉| 1000/1010 [03:52<00:02,  4.29it/s, loss=0.102, v_num=q6d8]
Validating: 0it [00:00, ?it/s]
Epoch 3:  99%|█████████▉| 1000/1010 [03:54<00:02,  4.26it/s, loss=0.105, v_num=q6d8]
Validating: 0it [00:00, ?it/s]
Epoch 4:  99%|█████████▉| 1000/1010 [03:56<00:02,  4.23it/s, loss=0.106, v_num=q6d8]
Validating: 0it [00:00, ?it/s]
Epoch 5:  99%|█████████▉| 1000/1010 [03:55<00:02,  4.25it/s, loss=0.108, v_num=q6d8]
Validating: 0it [00:00, ?it/s]
Epoch 6:  99%|█████████▉| 1000/1010 [03:53<00:02,  4.29it/s, loss=0.11, v_num=q6d8] 
Validating: 0it [00:00, ?it/s]
Epoch 7:  99%|█████████▉| 1000/1010 [03:55<00:02,  4.24it/s, loss=0.11, v_num=q6d8]
Validating: 0it [00:00, ?it/s]
Epoch 8:  99%|█████████▉| 1000/1010 [03:54<00:02,  4.27it/s, loss=0.111

## Initialize from trained model

In [4]:
import wandb
wandb.finish()

with open("dual_embedding_default.yaml") as f:
    hparams = yaml.load(f, Loader=yaml.FullLoader)
    
hparams["use_dual_encoder"] = True

model = VanillaDualEmbedding(hparams)

checkpoint = torch.load("/global/cfs/cdirs/m3443/usr/ryanliu/ITk_embedding/ITk_dual_embedding/3ijb4qnw/checkpoints/last.ckpt")
state_dict = checkpoint["state_dict"]
names = [i for i in state_dict]
for i in names:
    state = state_dict[i]
    i = i.replace("input_layer1", "input_layer2")
    i = i.replace("layers1", "layers2")
    i = i.replace("output_layer1", "output_layer2")
    state_dict[i] = state

model.load_state_dict(state_dict)
del state_dict

In [5]:
from pytorch_lightning.callbacks import ModelCheckpoint

checkpoint_callback = ModelCheckpoint(
    monitor='pur',
    mode="max",
    save_top_k=2,
    save_last=True)

In [ ]:
logger = WandbLogger(project="ITk_dual_embedding")
trainer = Trainer(gpus=1, max_epochs=hparams["max_epochs"], logger=logger, num_sanity_val_steps=2, callbacks=[checkpoint_callback], log_every_n_steps = 50, default_root_dir="/global/cfs/cdirs/m3443/usr/ryanliu/ITk_embedding/")
trainer.fit(model)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: exatrkx (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.11 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade



  | Name          | Type             | Params
---------------------------------------------------
0 | cos           | CosineSimilarity | 0     
1 | input_layer1  | Linear           | 12.3 K
2 | layers1       | ModuleList       | 5.2 M 
3 | output_layer1 | Linear           | 16.4 K
4 | input_layer2  | Linear           | 12.3 K
5 | layers2       | ModuleList       | 5.2 M 
6 | output_layer2 | Linear           | 16.4 K
7 | act           | GELU             | 0     
---------------------------------------------------
10.6 M    Trainable params
0         Non-trainable params
10.6 M    Total params
42.214    Total estimated model params size (MB)


Validation sanity check:   0%|          | 0/2 [00:00<?, ?it/s]

/global/homes/r/ryanliu/.conda/envs/gnn/lib/python3.8/site-packages/pytorch_lightning/trainer/data_loading.py:116: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 256 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Validation sanity check:  50%|█████     | 1/2 [00:01<00:01,  1.46s/it]

/global/homes/r/ryanliu/.conda/envs/gnn/lib/python3.8/site-packages/pytorch_lightning/utilities/data.py:59: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 5. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  warning_cache.warn(


/global/homes/r/ryanliu/.conda/envs/gnn/lib/python3.8/site-packages/pytorch_lightning/trainer/data_loading.py:116: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 256 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Epoch 0:  99%|█████████▉| 1000/1010 [03:52<00:02,  4.29it/s, loss=0.0123, v_num=uh1o]
Validating: 0it [00:00, ?it/s]
Epoch 1:  99%|█████████▉| 1000/1010 [04:04<00:02,  4.09it/s, loss=0.0115, v_num=uh1o]
Validating: 0it [00:00, ?it/s]
Epoch 2:  99%|█████████▉| 1000/1010 [04:16<00:02,  3.89it/s, loss=0.0114, v_num=uh1o]
Validating: 0it [00:00, ?it/s]
Epoch 3:  10%|▉         | 97/1010 [00:26<04:04,  3.73it/s, loss=0.0114, v_num=uh1o]  